**SPARK Big Data**
<br>Bu Collab notebookta apache spark ve big data işlemleri yapılacaktır.

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=507bb6431d2e0b638c4569d4919fa31d1851e648917c45481616c0f0669b7e8f
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark import SparkConf, SparkContext
import collections
sc=SparkContext()

In [ ]:
rdd=sc.parallelize([3,4,56,7.4,4.2])
sq=rdd.map(lambda x:x*x)
print(sq.collect())

[9, 16, 3136, 54.760000000000005, 17.64]


In [ ]:
sc.stop()

Ratings Histogram

In [ ]:
from pyspark import SparkConf, SparkContext
import collections
conf=SparkConf().setMaster('local').setAppName('RatingsHistogram') #
sc=SparkContext(conf=conf)
lines=sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/ml-100k/u.data')
ratings=lines.map(lambda x: x.split()[2])
result=ratings.countByValue()
sortedresults=collections.OrderedDict(sorted(result.items()))
for key,value in sortedresults.items():
  print(key,value)

1 6111
2 11370
3 27145
4 34174
5 21203


In [ ]:
sc.stop()

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
lines = sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/1800.csv')

In [ ]:
def parseline(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [ ]:
parsedLines = lines.map(parseline)
minTemps = parsedLines.filter(lambda x: 'TMIN' in x[1])
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
minTemps = stationTemps.reduceByKey(lambda x, y: min(x, y))


In [ ]:
results = minTemps.collect()
for result in results:
    print(result[0] + '\t{:.2f}F'.format(result[1]))

ITE00100554	5.36F
EZE00100082	7.70F


In [ ]:
sc.stop()

In [ ]:
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
lines = sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/1800.csv')
parsedLines = lines.map(parseline)
maxTemps = parsedLines.filter(lambda x: 'TMAX' in x[1])
stationTemps = maxTemps.map(lambda x: (x[0], x[2]))
maxTemps = stationTemps.reduceByKey(lambda x, y: max(x, y))
results = maxTemps.collect()
for result in results:
    print(result[0] + '\t{:.2f}F'.format(result[1]))
sc.stop()

ITE00100554	90.14F
EZE00100082	90.14F


In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
input = sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/book.txt')
words=input.flatMap(lambda x: x.split())
word_counts=words.countByValue() # kelimelerin frekanslarını bulur
for word,count in word_counts.items():
  print(word,count)
sc.stop()

In [ ]:
sc.stop()

https://medium.com/@sengonulserkan/apache-spark-word-count-a024c497fbb3

In [ ]:
import re
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
def normalizeWords(text):
  return re.compile(r'\W+', re.UNICODE).split(text.lower())

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
input = sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/book.txt')
words=input.flatMap(normalizeWords)

word_counts=words.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
word_counts_sorted=word_counts.map(lambda x: (x[1],x[0])).sortByKey(False)
results=word_counts_sorted.collect()

for result in results:
  count=str(result[0])
  word=result[1].encode('ascii', 'ignore')
  if (word):
    print(word.decode() + ":\t\t" + count)



In [ ]:
sc.stop()

In [ ]:
import re
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

def countCoOccurences(line):
  fields = line.split()
  return (int(fields[0]), len(fields)-1)

def parseNames(line):
  fields = line.split('\"')
  return (int(fields[0]), fields[1].encode('utf-8'))


names = sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/Marvel-names.txt')
namesRdd=names.map(parseNames)
lines=sc.textFile('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/Marvel-graph.txt')

pairings = lines.map(countCoOccurences)
totalFriendsByCharacter = pairings.reduceByKey(lambda x,y: x+y)
flipped = totalFriendsByCharacter.map(lambda xy: (xy[1], xy[0]))

mostPopular = flipped.max()
mostPopularName = namesRdd.lookup(mostPopular[1])[0]
print(str(mostPopularName) + " is the most popular superhero, with " + str(mostPopular[0]) + " co-appearances.")
#print top 5 popular name
for i in range(5):
  print(namesRdd.lookup(flipped.top(5)[i][1])[0])


b'CAPTAIN AMERICA' is the most popular superhero, with 1933 co-appearances.
b'CAPTAIN AMERICA'
b'SPIDER-MAN/PETER PAR'
b'IRON MAN/TONY STARK '
b'THING/BENJAMIN J. GR'
b'WOLVERINE/LOGAN '


In [ ]:
sc.stop()

In [ ]:
import re
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

sc = SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()

data=spark.read.csv('/content/drive/MyDrive/Bootcamp_projeler/day 10 Apache spark big data/pima-indians-diabetes.csv',header=True,inferSchema=True)
predictors=data.columns[:-1]
assembler=VectorAssembler(inputCols=predictors,outputCol='features')
data=assembler.transform(data).select('features','Outcome')

train_data,test_data=data.randomSplit([0.8,0.2], seed=42)
lr=LogisticRegression(featuresCol='features',labelCol='Outcome',maxIter=10)
model=lr.fit(train_data)
predictions=model.transform(test_data)

evaluator=BinaryClassificationEvaluator(labelCol='Outcome')
evaluator.evaluate(predictions)
print('Accuracy:',evaluator.evaluate(predictions))


Accuracy: 0.8840048840048842


In [ ]:
sc.stop()